## GoogleNet - InceptionV3


### Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import os

### Splitting Dataset

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

training_set = train_datagen.flow_from_directory('dataset',
                                                 target_size=(299, 299),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset='training')

validation_set = train_datagen.flow_from_directory('dataset',
                                                   target_size=(299, 299),
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   subset='validation')

### Training Model

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in base_model.layers[:40]:
    layer.trainable = False
for layer in base_model.layers[40:]:
    layer.trainable = True

new_classifier = Sequential([
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

cnn_model = Sequential([
    base_model,
    new_classifier 
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = cnn_model.fit(training_set, validation_data=validation_set, epochs=10)

### Extracting Image Features

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def extract_features_fixed(image_path, base_model):
    img = load_img(image_path, target_size=(299, 299)) 
    img_array = img_to_array(img) / 255.0  
    img_array = np.expand_dims(img_array, axis=0)  
    features = base_model.predict(img_array)
    return features.flatten()

def create_feature_list(dataset_dir, base_model):
    features_list = []
    image_paths = []
    
    for subdir, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_path = os.path.join(subdir, file)
                features = extract_features_fixed(file_path, base_model)
                features_list.append(features)
                image_paths.append(file_path)
    
    return np.array(features_list), image_paths

features_list, image_paths = create_feature_list("dataset", base_model)

### Finding Nearest Neighbors

In [ ]:
def find_nearest_images(image_path, base_model, n_neighbors=10):
    
    image_features = extract_features_fixed(image_path, base_model)
    
    neighbors = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    neighbors.fit(features_list)
    
    distances, indices = neighbors.kneighbors([image_features])
    
    print(f"Nearest images to {image_path}:")
    for i in range(n_neighbors):
        print(f"{image_paths[indices[0][i]]} - Distance: {distances[0][i]:.4f}")
    
    plt.figure(figsize=(12, 6))
    for i in range(n_neighbors):
        img = load_img(image_paths[indices[0][i]], target_size=(299, 299))
        plt.subplot(2, 5, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Distance: {distances[0][i]:.4f}")
    plt.show()

find_nearest_images('dataset/Motorbikes/image_0794.jpg', base_model, n_neighbors=10)
find_nearest_images('photos/plane.jpg', base_model, n_neighbors=10)